In [1]:
import pandas as pd
from pyexpat import features

data = pd.read_parquet('../cache/encoded_99q_scaled.parquet')
data = data.drop(columns=['pct_change_24h', 'pct_change_15min'])
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '../cache/encoded_99q_scaled.parquet'

In [2]:
data.dtypes

Timestamp           float64
Actor1Country         int64
Actor1GeoCountry      int64
Actor1Type            int64
Actor2Country         int64
Actor2GeoCountry      int64
Actor2Type            int64
ActionCountry         int64
EventType             int64
GoldsteinScale      float64
NumSources          float64
NumArticles         float64
AvgTone             float64
Magnitude           float64
Impact              float64
pct_change_30min    float64
dtype: object

In [3]:
# Split the data into train and test sets
train_data = data[data.index.year < 2023]
test_data = data[data.index.year == 2023]

print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)
smol_sample = train_data.tail(1000)
sample = train_data.tail(10000)
big_sample = train_data.tail(100000)
sample.head()

Train data shape: (4249654, 16)
Test data shape: (921887, 16)


,Timestamp,Actor1Country,Actor1GeoCountry,Actor1Type,Actor2Country,Actor2GeoCountry,Actor2Type,ActionCountry,EventType,GoldsteinScale,NumSources,NumArticles,AvgTone,Magnitude,Impact,pct_change_30min
Date,,,,,,,,,,,,,,,,
2022-12-27 01:00:00,1.161151,9,13,9,13,11,9,11,10,-0.523000,-0.677247,-0.106454,0.082086,-0.716820,-0.160537,0.000000
2022-12-27 01:00:00,1.161151,9,13,9,13,11,9,11,10,0.731167,-0.155088,-0.387892,-2.016639,1.902909,1.405324,0.000000
2022-12-27 01:00:00,1.161151,9,13,9,13,11,9,11,10,0.480334,-0.677247,-0.669330,-0.678348,0.073454,0.515962,0.000000
2022-12-27 01:15:00,1.161170,9,13,9,13,11,9,11,10,0.104084,-0.677247,-0.106454,0.405894,-1.108846,0.096066,-0.029688
2022-12-27 01:15:00,1.161170,2,4,9,13,3,9,4,10,0.480334,-0.155088,-0.387892,-0.706995,0.325471,0.580113,-0.029688


In [4]:
# cache the train and test data for later use
train_data.to_parquet('../cache/train_data.parquet')
test_data.to_parquet('../cache/test_data.parquet')

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Quantifies multicollinearity
vif_data = pd.DataFrame()
vif_data["Variable"] = sample.columns
vif_data["VIF"] = [variance_inflation_factor(sample.values, i) for i in range(sample.shape[1])]
print(vif_data)

            Variable          VIF
0          Timestamp  2044.879043
1      Actor1Country   193.521757
2   Actor1GeoCountry   165.545638
3         Actor1Type   877.957511
4      Actor2Country   213.618694
5   Actor2GeoCountry   146.489644
6         Actor2Type   775.696064
7      ActionCountry   194.181857
8          EventType    15.299656
9     GoldsteinScale     4.798859
10        NumSources     1.333725
11       NumArticles     1.303256
12           AvgTone     1.862013
13         Magnitude     2.340180
14            Impact     4.974763
15  pct_change_30min     1.005640


### Vector Autoregression (VAR)

In [6]:
import statsmodels.api as sm
from statsmodels.tsa.api import VAR

model = VAR(big_sample)
var_results = model.fit(maxlags=15, ic='aic') # Fit with automatic lag order selection based on AIC
lag_order = var_results.k_ar
predictions = var_results.forecast(sample.values[-lag_order:], steps=5) # Forecast 5 steps ahead
print(predictions)

/opt/homebrew/anaconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


[[ 1.17029975e+00  8.94934196e+00  1.27236856e+01  8.96695347e+00
   1.30348167e+01  1.09387702e+01  8.97698238e+00  1.07606560e+01
   9.69531217e+00  1.21975588e-02 -2.79608514e-01 -1.51786627e-01
   2.22791905e-01 -3.01620065e-01  2.03298185e-03 -1.82967353e-02]
 [ 1.17030088e+00  8.96032483e+00  1.27255447e+01  8.99521480e+00
   1.29816243e+01  1.09779270e+01  8.97662315e+00  1.07743490e+01
   9.89225659e+00  4.79441676e-02 -2.24129669e-01 -6.87135596e-02
   1.79035586e-01 -1.90289034e-01  6.15890685e-02 -1.98621729e-02]
 [ 1.17030214e+00  8.99202937e+00  1.26400020e+01  8.99073216e+00
   1.30161442e+01  1.09952493e+01  8.98270102e+00  1.07330021e+01
   9.92722907e+00 -2.78012930e-03 -2.50177419e-01 -1.62511501e-01
   2.30497986e-01 -2.70455440e-01 -4.80875984e-03 -1.89104160e-02]
 [ 1.17030333e+00  8.97824926e+00  1.26386877e+01  8.98713445e+00
   1.30196361e+01  1.09865047e+01  8.97757474e+00  1.07181917e+01
   9.87856510e+00 -1.57157516e-02 -2.09153073e-01 -1.47691715e-01
   1.63

### dynamic factor model (DFM)

In [7]:
from statsmodels.tsa.statespace.dynamic_factor import DynamicFactor

# Fit a dynamic factor model
model = DynamicFactor(sample, k_factors=1, factor_order=1)
dfm_results = model.fit()
print(dfm_results.summary())

/opt/homebrew/anaconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           33     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.10727D+05    |proj g|=  8.75607D+07


 This problem is unconstrained.



At iterate    5    f=  4.22851D+02    |proj g|=  2.77795D+02

At iterate   10    f=  1.00869D+02    |proj g|=  1.04601D+01

At iterate   15    f=  4.62948D+01    |proj g|=  1.41803D+00

At iterate   20    f=  4.31373D+01    |proj g|=  1.26693D+00

At iterate   25    f=  4.27278D+01    |proj g|=  6.49650D-01

At iterate   30    f=  4.27175D+01    |proj g|=  2.83374D-02

At iterate   35    f=  4.27114D+01    |proj g|=  1.02908D-01

At iterate   40    f=  4.26858D+01    |proj g|=  1.50243D-01

At iterate   45    f=  4.26830D+01    |proj g|=  1.70440D-02

At iterate   50    f=  4.26810D+01    |proj g|=  1.56699D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tn

/opt/homebrew/anaconda3/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                                                                                                                                             Statespace Model Results                                                                                                                                             
Dep. Variable:     ['Timestamp', 'Actor1Country', 'Actor1GeoCountry', 'Actor1Type', 'Actor2Country', 'Actor2GeoCountry', 'Actor2Type', 'ActionCountry', 'EventType', 'GoldsteinScale', 'NumSources', 'NumArticles', 'AvgTone', 'Magnitude', 'Impact', 'pct_change_30min']   No. Observations:                10000
Model:                                                                                                                                                                                                                                  DynamicFactor(factors=1, order=1)   Log Likelihood             -426809.686
Date:                                                                          

In [8]:
forecast_steps = 5
forecast = dfm_results.get_forecast(steps=forecast_steps)
# Extract the predicted values from the forecast result
predicted_values = forecast.predicted_mean
# Or for the full prediction including uncertainty (confidence intervals)
prediction_conf_int = forecast.conf_int()

# Display the predicted values and the confidence intervals
print("Predicted Values:")
print(predicted_values)

print("\nConfidence Intervals for Predictions:")
print(prediction_conf_int)

Predicted Values:
          Timestamp  Actor1Country  Actor1GeoCountry    Actor1Type  \
10000  1.317899e-05   1.908003e-03      7.350477e-03  3.969926e-05   
10001  1.153344e-06   1.669766e-04      6.432684e-04  3.474235e-06   
10002  1.009336e-07   1.461277e-05      5.629489e-05  3.040436e-07   
10003  8.833086e-09   1.278819e-06      4.926582e-06  2.660802e-08   
10004  7.730173e-10   1.119144e-07      4.311441e-07  2.328570e-09   

       Actor2Country  Actor2GeoCountry    Actor2Type  ActionCountry  \
10000   1.758421e-03      3.562843e-03  1.098794e-04   6.078165e-03   
10001   1.538862e-04      3.117980e-04  9.615967e-06   5.319236e-04   
10002   1.346717e-05      2.728664e-05  8.415302e-07   4.655067e-05   
10003   1.178564e-06      2.387959e-06  7.364554e-08   4.073828e-06   
10004   1.031406e-07      2.089794e-07  6.445004e-09   3.565163e-07   

          EventType  GoldsteinScale    NumSources   NumArticles       AvgTone  \
10000  1.696200e-04    9.801779e-03 -3.353194e-03 -3.

/opt/homebrew/anaconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/homebrew/anaconda3/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


### Vector Autoregression Moving-Average (VARMA)

In [9]:
from statsmodels.tsa.statespace.varmax import VARMAX

model = VARMAX(smol_sample, order=(1, 1))  # (p, q) - autoregressive and moving average orders
varmax_results = model.fit(disp=False)
predictions = varmax_results.forecast(steps=5)
print(predictions)

In [10]:
# cache the models for later use
import joblib
joblib.dump(var_results, '../cache/var_model.joblib')
joblib.dump(dfm_results, '../cache/dfm_model.joblib')
joblib.dump(varmax_results, '../cache/varmax_model.joblib')

NameError: name 'varmax_results' is not defined

In [11]:
# purely for testing
# Load the models from cache
var_results_loaded = joblib.load('../cache/var_model.joblib')
print("done")
var_results_loaded.forecast(sample.values[var_results_loaded.k_ar:], steps=5)

KeyError: -1